In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('litigation.csv')

In [ ]:
df['id'] = df.groupby('country').ngroup()

In [ ]:
import numpy as np
df['lnghg'] = np.log(df['ghg'])
df['lngdp'] = np.log(df['gdp'])
df['lngdp2'] = df['lngdp']**2
df['lnpopulation'] = np.log(df['population'])
df['lnlaw'] = np.log(df['ruleoflaw'])

In [ ]:
df[['lnghg', 'lngdp', 'lngdp2', 'lnpopulation', 'lnlaw', 'litigation']].describe().transpose()

### TABLE 3 NEW

In [ ]:
# reg lnghg lngdp lngdp2 lnpopulation lnlaw litigation , robust

import statsmodels.api as sm

data = df[['lnghg', 'lngdp', 'lngdp2', 'lnpopulation', 
           'lnlaw', 'litigation']].dropna()
X = data[['lngdp', 'lngdp2', 'lnpopulation', 'lnlaw', 'litigation']]
X = sm.add_constant(X)
y = data['lnghg']
model = sm.OLS(y, X).fit(cov_type='HC3')
print(model.summary())

In [ ]:
# xtreg lnghg lngdp lngdp2 lnpopulation lnlaw litigation i.year, fe robust

from linearmodels.panel import PanelOLS

data = df[['lnghg', 'lngdp', 'lngdp2', 'lnpopulation', 
           'lnlaw', 'litigation', 'year', 'id']].dropna()
data['cons'] = 1
panel_data = data.set_index(['id', 'year'])

y = panel_data['lnghg']
X = panel_data[['lngdp', 'lngdp2', 'lnpopulation', 'lnlaw', 'litigation', 'cons']]

model = PanelOLS(y, X, entity_effects=True, time_effects=True)
results = model.fit(cov_type='kernel', kernel='bartlett')
print(results)

In [ ]:
hi_countries = ['BEL', 'AUS', 'CAN', 'CHE', 'DEU', 'ESP', 'EST', 'FIN', 
                'FRA', 'GBR', 'GRC', 'HRV', 'ITA', 'LTU', 'LUX', 'NLD', 
                'NOR', 'NZL', 'POL', 'PRT', 'SVK', 'SWE', 'ROU']
df['HI'] = 0
df.loc[df['country'].isin(hi_countries), 'HI'] = 1

In [ ]:
# reg lnghg lngdp lngdp2 lnpopulation lnlaw HI#c.litigation, robust

data = df[['lnghg', 'lngdp', 'lngdp2', 'lnpopulation', 
           'lnlaw', 'litigation', 'year', 'id', 'HI']].dropna()
data['HITRUE_litigation'] = data['HI'] * data['litigation']
data['HIFALSE_litigation'] = (1-data['HI']) * data['litigation']

X = data[['lngdp', 'lngdp2', 'lnpopulation', 'lnlaw', 'HITRUE_litigation', 'HIFALSE_litigation']]
X = sm.add_constant(X)
y = data['lnghg']

model = sm.OLS(y, X).fit(cov_type='HC3')
print(model.summary())

In [ ]:
# xtreg lnghg lngdp lngdp2 lnpopulation lnlaw HI#c.litigation i.year, fe robust

data = df[['lnghg', 'lngdp', 'lngdp2', 'lnpopulation', 
           'lnlaw', 'litigation', 'year', 'id', 'HI']].dropna()
data['cons'] = 1
data['HITRUE_litigation'] = data['HI'] * data['litigation']
data['HIFALSE_litigation'] = (1-data['HI']) * data['litigation']
panel_data = data.set_index(['id', 'year'])

y = panel_data['lnghg']
X = panel_data[['lngdp', 'lngdp2', 'lnpopulation', 'lnlaw', 'HITRUE_litigation', 'HIFALSE_litigation', 'cons']]

model = PanelOLS(y, X, entity_effects=True, time_effects=True)
results = model.fit(cov_type='kernel', kernel='bartlett')
print(results)